In [1]:
import arcpy, os
from arcpy import env
from arcpy.sa import *

In [2]:
arcpy.env.overwriteOutput = True

In [3]:
#arcpy.CheckExtension("Spatial Analyst")
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [4]:
env.workspace = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data\ForAppendixF\NaturalHazards\OSHD-1_GIS-bundle\OSHD Release 1_0.gdb'

In [5]:
# reclassify
outReclass1 = Reclassify("Dry_Landslide_Susceptibility_Map", "Value", 
                         RemapRange([[0,4,0],[5,9,1]]), "NODATA")
outReclass1.save("Dry_Landslide_CLMPO_Reclass")

In [6]:
outReclass2 = Reclassify("Wet_Landslide_Susceptibility_Map", "Value", 
                         RemapRange([[0,5,0],[6,10,1]]), "NODATA")
outReclass2.save("Wet_Landslide_CLMPO_Reclass")

In [7]:
outReclass3 = Reclassify("Liquefaction_Susceptibility_Map", "Value", 
                         RemapRange([[0,2,0],[3,5,1]]), "NODATA")
outReclass3.save("Liquefaction_CLMPO_Reclass")

In [8]:
outReclass4 = Reclassify("Probability_of_Damaging_Shaking_Map", "Value", 
                         RemapRange([[0,3,0],[4,5,1]]), "NODATA")
outReclass4.save("Damaging_Shaking_CLMPO_Reclass")

In [9]:
# raster to polygon
arcpy.RasterToPolygon_conversion("Dry_Landslide_CLMPO_Reclass", "Dry_Landslide_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Dry_Landslide_CLMPO_Reclass_Polygon'>

In [10]:
arcpy.RasterToPolygon_conversion("Wet_Landslide_CLMPO_Reclass", "Wet_Landslide_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Wet_Landslide_CLMPO_Reclass_Polygon'>

In [11]:
arcpy.RasterToPolygon_conversion("Liquefaction_CLMPO_Reclass", "Liquefaction_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Liquefaction_CLMPO_Reclass_Polygon'>

In [12]:
arcpy.RasterToPolygon_conversion("Damaging_Shaking_CLMPO_Reclass", "Damaging_Shaking_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Damaging_Shaking_CLMPO_Reclass_Polygon'>

In [13]:
shpfiles = ['Dry_Landslide', 'Wet_Landslide', 'Liquefaction', 'Damaging_Shaking']

In [14]:
# merge
for shpfile in shpfiles:
    fc = 'hazard_layer'
    arcpy.MakeFeatureLayer_management(shpfile+'_CLMPO_Reclass_Polygon', fc)
    arcpy.SelectLayerByAttribute_management(fc, 'NEW_SELECTION', '"gridcode"=1')
    #arcpy.CopyFeatures_management('hazard_layer', shpfile)
    field = 'source'
    arcpy.AddField_management(fc, field,  "TEXT", "", "", 50)
    with arcpy.da.UpdateCursor(fc, field) as cursor:
        for row in cursor:
            row[0] = shpfile
            cursor.updateRow(row)
    if shpfile == 'Dry_Landslide':
        arcpy.CopyFeatures_management(fc, 'earthquake_layer')
    else:
        arcpy.Append_management(fc, 'earthquake_layer', "TEST","","")

In [15]:
outpath = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data\ForAppendixF'

In [16]:
# dissolve
arcpy.Dissolve_management('earthquake_layer', os.path.join(outpath, 'NaturalHazards', 'OregonEarthquakeLayer.shp'), 'source')

ExecuteError: ERROR 160196: Invalid Topology
Failed to execute (Dissolve).
